# Training a microWakeWord Model

This notebook steps you through training a basic microWakeWord model. It is intended as a **starting point** for advanced users. You should use Python 3.10.

**The model generated will most likely not be usable for everyday use; it may be difficult to trigger or falsely activates too frequently. You will most likely have to experiment with many different settings to obtain a decent model!**

In the comment at the start of certain blocks, I note some specific settings to consider modifying.

This runs on Google Colab, but is extremely slow compared to training on a local GPU. If you must use Colab, be sure to Change the runtime type to a GPU. Even then, it still slow!

At the end of this notebook, you will be able to download a tflite file. To use this in ESPHome, you need to write a model manifest JSON file. See the [ESPHome documentation](https://esphome.io/components/micro_wake_word) for the details and the [model repo](https://github.com/esphome/micro-wake-word-models/tree/main/models/v2) for examples.

In [ ]:
!pip install torchcodec

In [ ]:
# Installs microWakeWord. Be sure to restart the session after this is finished.
import platform

if platform.system() == "Darwin":
    # `pymicro-features` is installed from a fork to support building on macOS
    !pip install 'git+https://github.com/puddly/pymicro-features@puddly/minimum-cpp-version'

# `audio-metadata` is installed from a fork to unpin `attrs` from a version that breaks Jupyter
!pip install 'git+https://github.com/whatsnowplaying/audio-metadata@d4ebb238e6a401bb1a5aaaac60c9e2b3cb30929f'

!git clone https://github.com/kahrendt/microWakeWord
!pip install -e ./microWakeWord

In [ ]:
# Generates 1 sample of the target word for manual verification.

import os
import sys
import platform

from IPython.display import Audio

!rm -rf ./piper-sample-generator

if not os.path.exists("./piper-sample-generator"):
    if platform.system() == "Darwin":
        !git clone -b mps-support https://github.com/kahrendt/piper-sample-generator
    else:
        !git clone https://github.com/KPEKEP/piper-sample-generator

    !wget -O piper-sample-generator/models/en_US-libritts_r-medium.pt 'https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/en_US-libritts_r-medium.pt'

    # Install system dependencies
    !pip install torch torchaudio piper-phonemize-cross==1.2.1

    if "piper-sample-generator/" not in sys.path:
        sys.path.append("piper-sample-generator/")

In [ ]:
target_word = "ˈsɜrjvɪtorj"

!python3 piper-sample-generator/generate_samples.py "{target_word}" \
--phoneme \
--max-samples 1 \
--batch-size 1 \
--output-dir generated_samples \
--verbose

Audio("generated_samples/0.wav", autoplay=True)

In [ ]:
# Generates a larger amount of wake word samples.
# Start here when trying to improve your model.
# See https://github.com/rhasspy/piper-sample-generator for the full set of
# parameters. In particular, experiment with noise-scales and noise-scale-ws,
# generating negative samples similar to the wake word, and generating many more
# wake word samples, possibly with different phonetic pronunciations.

!python3 piper-sample-generator/generate_samples.py "{target_word}" \
--phoneme \
--max-samples 20000 \
--batch-size 100 \
--output-dir generated_samples

In [ ]:
# Generate sound-a-like words for negative testing

!python3 piper-sample-generator/generate_samples.py "conservator" \
--max-samples 10000 \
--batch-size 100 \
--output-dir negative_genrated_samples_1

!python3 piper-sample-generator/generate_samples.py "observator" \
--max-samples 10000 \
--batch-size 100 \
--output-dir negative_genrated_samples_2

!python3 piper-sample-generator/generate_samples.py "server" \
--max-samples 10000 \
--batch-size 100 \
--output-dir negative_genrated_samples_3

In [ ]:
# Downloads audio data for augmentation. This can be slow!
# Borrowed from openWakeWord's automatic_model_training.ipynb, accessed March 4, 2024
#
# **Important note!** The data downloaded here has a mixture of difference
# licenses and usage restrictions. As such, any custom models trained with this
# data should be considered as appropriate for **non-commercial** personal use only.

import datasets
import scipy
import os
import shutil
import librosa
import numpy as np

from pathlib import Path
from tqdm import tqdm

## Download MIR RIR data

shutil.rmtree("./mit_rirs", ignore_errors=True)
shutil.rmtree("./audioset", ignore_errors=True)
shutil.rmtree("./fma", ignore_errors=True)

output_dir = "./mit_rirs"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
    # Load dataset with explicit Audio feature that doesn't use torchcodec
    rir_dataset = datasets.load_dataset(
        "davidscripka/MIT_environmental_impulse_responses", 
        split="train", 
        streaming=True
    ).cast_column("audio", datasets.Audio(decode=True))
    
    # Process the data
    for i, row in enumerate(tqdm(rir_dataset)):
        try:
            # Try to access audio data
            audio_data = row['audio']
            audio_array = audio_data['array']
            sample_rate = audio_data['sampling_rate']
            
            # Generate filename
            name = f"rir_audio_{i:06d}.wav"
            
            # Resample if needed
            if sample_rate != 16000:
                import librosa
                audio_array = librosa.resample(audio_array, orig_sr=sample_rate, target_sr=16000)
            
            # Convert to 16-bit PCM
            audio_int16 = (audio_array * 32767).astype(np.int16)
            
            # Save as wav file
            scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, audio_int16)
            
        except Exception as e:
            print(f"Error processing row {i}: {e}")
            continue

## Download noise and background audio

# Audioset Dataset (https://research.google.com/audioset/dataset/index.html)
# Download one part of the audioset .tar files, extract, and convert to 16khz
# For full-scale training, it's recommended to download the entire dataset from
# https://huggingface.co/datasets/agkphysics/AudioSet, and
# even potentially combine it with other background noise datasets (e.g., FSD50k, Freesound, etc.)

if not os.path.exists("audioset"):
    os.mkdir("audioset")

    fname = "bal_train09.tar"
    out_dir = f"audioset/{fname}"
    link = "https://huggingface.co/datasets/agkphysics/AudioSet/resolve/main/data/" + fname
    !wget -O {out_dir} {link}
    !cd audioset && tar -xf bal_train09.tar

    output_dir = "./audioset_16k"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    # Use librosa directly instead of datasets Audio to avoid torchcodec issues
    audio_files = list(Path("audioset/audio").glob("**/*.flac"))
    for audio_file in tqdm(audio_files):
        try:
            # Load audio with librosa
            audio_array, sample_rate = librosa.load(str(audio_file), sr=16000)
            
            # Generate output filename
            name = audio_file.name.replace(".flac", ".wav")
            
            # Convert to 16-bit PCM
            audio_int16 = (audio_array * 32767).astype(np.int16)
            
            # Save as wav file
            scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, audio_int16)
            
        except Exception as e:
            print(f"Error processing {audio_file}: {e}")
            continue

# Free Music Archive dataset
# https://github.com/mdeff/fma
# (Third-party mchl914 extra small set)

output_dir = "./fma"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    fname = "fma_xs.zip"
    link = "https://huggingface.co/datasets/mchl914/fma_xsmall/resolve/main/" + fname
    out_dir = f"fma/{fname}"
    !wget -O {out_dir} {link}
    !cd {output_dir} && unzip -q {fname}

    output_dir = "./fma_16k"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    # Use librosa directly instead of datasets Audio to avoid torchcodec issues
    audio_files = list(Path("fma/fma_small").glob("**/*.mp3"))
    for audio_file in tqdm(audio_files):
        try:
            # Load audio with librosa
            audio_array, sample_rate = librosa.load(str(audio_file), sr=16000)
            
            # Generate output filename
            name = audio_file.name.replace(".mp3", ".wav")
            
            # Convert to 16-bit PCM
            audio_int16 = (audio_array * 32767).astype(np.int16)
            
            # Save as wav file
            scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, audio_int16)
            
        except Exception as e:
            print(f"Error processing {audio_file}: {e}")
            continue

In [ ]:
# Sets up the augmentations.
# To improve your model, experiment with these settings and use more sources of
# background clips.

from microwakeword.audio.augmentation import Augmentation
from microwakeword.audio.clips import Clips
from microwakeword.audio.spectrograms import SpectrogramGeneration

clips = Clips(input_directory='generated_samples',
              file_pattern='*.wav',
              max_clip_duration_s=None,
              remove_silence=False,
              random_split_seed=10,
              split_count=0.1,
              )
augmenter = Augmentation(augmentation_duration_s=3.2,
                         augmentation_probabilities = {
                                "SevenBandParametricEQ": 0.1,
                                "TanhDistortion": 0.1,
                                "PitchShift": 0.1,
                                "BandStopFilter": 0.1,
                                "AddColorNoise": 0.1,
                                "AddBackgroundNoise": 0.75,
                                "Gain": 1.0,
                                "RIR": 0.5,
                            },
                         impulse_paths = ['mit_rirs'],
                         background_paths = ['fma_16k', 'audioset_16k'],
                         background_min_snr_db = -5,
                         background_max_snr_db = 10,
                         min_jitter_s = 0.195,
                         max_jitter_s = 0.205,
                         )


In [5]:
from microwakeword.audio.augmentation import Augmentation
from microwakeword.audio.clips import Clips
from microwakeword.audio.spectrograms import SpectrogramGeneration

negative_clips = []
negative_augmenters = []
for i in range(1,4):
    neg_clips = Clips(input_directory='negative_genrated_samples_'+str(i),
                  file_pattern='*.wav',
                  max_clip_duration_s=None,
                  remove_silence=False,
                  random_split_seed=10,
                  split_count=0.1,
                  )
    neg_augmenter = Augmentation(augmentation_duration_s=3.2,
                             augmentation_probabilities = {
                                    "SevenBandParametricEQ": 0.1,
                                    "TanhDistortion": 0.1,
                                    "PitchShift": 0.1,
                                    "BandStopFilter": 0.1,
                                    "AddColorNoise": 0.1,
                                    "AddBackgroundNoise": 0.75,
                                    "Gain": 1.0,
                                    "RIR": 0.5,
                                },
                             impulse_paths = ['mit_rirs'],
                             background_paths = ['fma_16k', 'audioset_16k'],
                             background_min_snr_db = -5,
                             background_max_snr_db = 10,
                             min_jitter_s = 0.195,
                             max_jitter_s = 0.205,
                             )
    negative_clips.append(neg_clips)
    negative_augmenters.append(neg_augmenter)

In [ ]:
# Augment a random clip and play it back to verify it works well

from IPython.display import Audio
from microwakeword.audio.audio_utils import save_clip

random_clip = clips.get_random_clip()
augmented_clip = augmenter.augment_clip(random_clip)
save_clip(augmented_clip, 'augmented_clip.wav')

Audio("augmented_clip.wav", autoplay=True)

In [ ]:
# Augment samples and save the training, validation, and testing sets.
# Validating and testing samples generated the same way can make the model
# benchmark better than it performs in real-word use. Use real samples or TTS
# samples generated with a different TTS engine to potentially get more accurate
# benchmarks.

import os
from mmap_ninja.ragged import RaggedMmap

output_dir = 'generated_augmented_features'

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

splits = ["training", "validation", "testing"]
for split in splits:
  out_dir = os.path.join(output_dir, split)
  if not os.path.exists(out_dir):
      os.mkdir(out_dir)


  split_name = "train"
  repetition = 2

  spectrograms = SpectrogramGeneration(clips=clips,
                                     augmenter=augmenter,
                                     slide_frames=10,    # Uses the same spectrogram repeatedly, just shifted over by one frame. This simulates the streaming inferences while training/validating in nonstreaming mode.
                                     step_ms=10,
                                     )
  if split == "validation":
    split_name = "validation"
    repetition = 1
  elif split == "testing":
    split_name = "test"
    repetition = 1
    spectrograms = SpectrogramGeneration(clips=clips,
                                     augmenter=augmenter,
                                     slide_frames=1,    # The testing set uses the streaming version of the model, so no artificial repetition is necessary
                                     step_ms=10,
                                     )

  RaggedMmap.from_generator(
      out_dir=os.path.join(out_dir, 'wakeword_mmap'),
      sample_generator=spectrograms.spectrogram_generator(split=split_name, repeat=repetition),
      batch_size=100,
      verbose=True,
  )

In [7]:
import os
from mmap_ninja.ragged import RaggedMmap

output_dir = 'negative_generated_augmented_features'



splits = ["training", "validation", "testing"]
for split in splits:
    for i in range(1,4):
        numbered_dir = output_dir+"_"+str(i)
        if not os.path.exists(numbered_dir):
            os.mkdir(numbered_dir)        
        out_dir = os.path.join(numbered_dir, split)
        if not os.path.exists(out_dir):
            os.mkdir(out_dir)
        
        
        split_name = "train"
        repetition = 2
        
        spectrograms = SpectrogramGeneration(clips=negative_clips[i-1],
                                         augmenter=negative_augmenters[i-1],
                                         slide_frames=10,    # Uses the same spectrogram repeatedly, just shifted over by one frame. This simulates the streaming inferences while training/validating in nonstreaming mode.
                                         step_ms=10,
                                         )
        if split == "validation":
            split_name = "validation"
            repetition = 1
        elif split == "testing":
            split_name = "test"
            repetition = 1
            spectrograms = SpectrogramGeneration(clips=negative_clips[i-1],
                                             augmenter=negative_augmenters[i-1],
                                             slide_frames=1,    # The testing set uses the streaming version of the model, so no artificial repetition is necessary
                                             step_ms=10,
                                             )
            
        RaggedMmap.from_generator(
          out_dir=os.path.join(out_dir, 'wakeword_mmap'),
          sample_generator=spectrograms.spectrogram_generator(split=split_name, repeat=repetition),
          batch_size=100,
          verbose=True,
        )

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
# Downloads pre-generated spectrogram features (made for microWakeWord in
# particular) for various negative datasets. This can be slow!

output_dir = './negative_datasets'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    link_root = "https://huggingface.co/datasets/kahrendt/microwakeword/resolve/main/"
    filenames = ['dinner_party.zip', 'dinner_party_eval.zip', 'no_speech.zip', 'speech.zip']
    for fname in filenames:
        link = link_root + fname

        zip_path = f"negative_datasets/{fname}"
        !wget -O {zip_path} {link}
        !unzip -q {zip_path} -d {output_dir}

In [1]:
# Save a yaml config that controls the training process
# These hyperparamters can make a huge different in model quality.
# Experiment with sampling and penalty weights and increasing the number of
# training steps.

import yaml
import os

config = {}

config["window_step_ms"] = 10

config["train_dir"] = (
    "trained_models/wakeword"
)


# Each feature_dir should have at least one of the following folders with this structure:
#  training/
#    ragged_mmap_folders_ending_in_mmap
#  testing/
#    ragged_mmap_folders_ending_in_mmap
#  testing_ambient/
#    ragged_mmap_folders_ending_in_mmap
#  validation/
#    ragged_mmap_folders_ending_in_mmap
#  validation_ambient/
#    ragged_mmap_folders_ending_in_mmap
#
#  sampling_weight: Weight for choosing a spectrogram from this set in the batch
#  penalty_weight: Penalizing weight for incorrect predictions from this set
#  truth: Boolean whether this set has positive samples or negative samples
#  truncation_strategy = If spectrograms in the set are longer than necessary for training, how are they truncated
#       - random: choose a random portion of the entire spectrogram - useful for long negative samples
#       - truncate_start: remove the start of the spectrogram
#       - truncate_end: remove the end of the spectrogram
#       - split: Split the longer spectrogram into separate spectrograms offset by 100 ms. Only for ambient sets

config["features"] = [
    {
        "features_dir": "generated_augmented_features",
        "sampling_weight": 2.0,
        "penalty_weight": 1.0,
        "truth": True,
        "truncation_strategy": "truncate_start",
        "type": "mmap",
    },
    {
        "features_dir": "negative_generated_augmented_features_1",
        "sampling_weight": 10.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "random",
        "type": "mmap",
    },
    {
        "features_dir": "negative_generated_augmented_features_2",
        "sampling_weight": 10.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "random",
        "type": "mmap",
    },    
    {
        "features_dir": "negative_generated_augmented_features_3",
        "sampling_weight": 10.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "random",
        "type": "mmap",
    },        
    {
        "features_dir": "negative_datasets/speech",
        "sampling_weight": 10.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "random",
        "type": "mmap",
    },
    {
        "features_dir": "negative_datasets/dinner_party",
        "sampling_weight": 10.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "random",
        "type": "mmap",
    },
    {
        "features_dir": "negative_datasets/no_speech",
        "sampling_weight": 5.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "random",
        "type": "mmap",
    },
    { # Only used for validation and testing
        "features_dir": "negative_datasets/dinner_party_eval",
        "sampling_weight": 0.0,
        "penalty_weight": 1.0,
        "truth": False,
        "truncation_strategy": "split",
        "type": "mmap",
    },
]

# Number of training steps in each iteration - various other settings are configured as lists that corresponds to different steps
config["training_steps"] = [2000] # [10000]

# Penalizing weight for incorrect class predictions - lists that correspond to training steps
config["positive_class_weight"] = [1]
config["negative_class_weight"] = [20]

config["learning_rates"] = [
    0.001,
]  # Learning rates for Adam optimizer - list that corresponds to training steps
config["batch_size"] = 128

config["time_mask_max_size"] = [
    0
]  # SpecAugment - list that corresponds to training steps
config["time_mask_count"] = [0]  # SpecAugment - list that corresponds to training steps
config["freq_mask_max_size"] = [
    0
]  # SpecAugment - list that corresponds to training steps
config["freq_mask_count"] = [0]  # SpecAugment - list that corresponds to training steps

config["eval_step_interval"] = (
    500  # Test the validation sets after every this many steps
)
config["clip_duration_ms"] = (
    2100  # Maximum length of wake word that the streaming model will accept
)

# The best model weights are chosen first by minimizing the specified minimization metric below the specified target_minimization
# Once the target has been met, it chooses the maximum of the maximization metric. Set 'minimization_metric' to None to only maximize
# Available metrics:
#   - "loss" - cross entropy error on validation set
#   - "accuracy" - accuracy of validation set
#   - "recall" - recall of validation set
#   - "precision" - precision of validation set
#   - "false_positive_rate" - false positive rate of validation set
#   - "false_negative_rate" - false negative rate of validation set
#   - "ambient_false_positives" - count of false positives from the split validation_ambient set
#   - "ambient_false_positives_per_hour" - estimated number of false positives per hour on the split validation_ambient set
config["target_minimization"] = 0.9
config["minimization_metric"] = None  # Set to None to disable

config["maximization_metric"] = "average_viable_recall"

with open(os.path.join("training_parameters.yaml"), "w") as file:
    documents = yaml.dump(config, file)

In [1]:
# Trains a model. When finished, it will quantize and convert the model to a
# streaming version suitable for on-device detection.
# It will resume if stopped, but it will start over at the configured training
# steps in the yaml file.
# Change --train 0 to only convert and test the best-weighted model.
# On Google colab, it doesn't print the mini-batch results, so it may appear
# stuck for several minutes! Additionally, it is very slow compared to training
# on a local GPU.

!CUDA_VISIBLE_DEVICES=-1 python -m microwakeword.model_train_eval \
--training_config='training_parameters.yaml' \
--train 1 \
--restore_checkpoint 0 \
--test_tf_nonstreaming 0 \
--test_tflite_nonstreaming 0 \
--test_tflite_nonstreaming_quantized 0 \
--test_tflite_streaming 0 \
--test_tflite_streaming_quantized 1 \
--use_weights "best_weights" \
mixednet \
--pointwise_filters "64,64,64,64" \
--repeat_in_block  "1, 1, 1, 1" \
--mixconv_kernel_sizes '[5], [7,11], [9,15], [23]' \
--residual_connection "0,0,0,0" \
--first_conv_filters 32 \
--first_conv_kernel_size 5 \
--stride 2

2025-07-25 17:41:07.663276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753458067.681765    6447 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753458067.687100    6447 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753458067.701350    6447 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753458067.701375    6447 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753458067.701379    6447 computation_placer.cc:177] computation placer alr

IOStream.flush timed out
IOStream.flush timed out


INFO:absl:Step 500 (nonstreaming): Validation: recall at no faph = 0.000 with cutoff 1.00, accuracy = 96.89%, recall = 93.60%, precision = 83.20%, ambient false positives = 601, estimated false positives per hour = 62.14793, loss = 0.02510, auc = 0.99407, average viable recall = 0.620802704
INFO:absl:Sharding callback duration: 75 microseconds
INFO:absl:So far the best minimization quantity is 0.000 with best maximization quantity of 62.08027%; no faph cutoff is 1.00
INFO:absl:Step #1000: rate 0.001000, accuracy 99.61%, recall 92.90%, precision 95.99%, cross entropy 0.012162
2025-07-25 18:48:55.316729: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5791651840 exceeds 10% of free system memory.
INFO:absl:Step 1000 (nonstreaming): Validation: recall at no faph = 0.000 with cutoff 1.00, accuracy = 95.99%, recall = 97.62%, precision = 76.34%, ambient false positives = 655, estimated false positives per hour = 67.73193, loss = 0.02868, auc = 0.99720, average 

In [ ]:
# Downloads the tflite model file. To use on the device, you need to write a
# Model JSON file. See https://esphome.io/components/micro_wake_word for the
# documentation and
# https://github.com/esphome/micro-wake-word-models/tree/main/models/v2 for
# examples. Adjust the probability threshold based on the test results obtained
# after training is finished. You may also need to increase the Tensor arena
# model size if the model fails to load.--verbosity

from google.colab import files

files.download(f"trained_models/wakeword/tflite_stream_state_internal_quant/stream_state_internal_quant.tflite")